# Revisiting Food-Safety Inspections from the Chicago Dataset - A Tutorial (Part 2)

## 1. In Review

## 2. Food Inspection Features

In [ ]:
import numpy as np
import pandas as pd
import os.path

root_path = os.path.dirname(os.getcwd())

# Load food inspection data
inspections = pd.read_csv(os.path.join(root_path, "DATA/food_inspections.csv"))

# Create basis for model_data
data = inspections.loc[:, ["inspection_id", "license", "inspection_date", "facility_type"]]

### 2.1. Pass / Fail Flags

In [ ]:
# Create pass / fail flags
data["pass_flag"] = inspections.results.apply(lambda x: 1 if x == "Pass" else 0)
data["fail_flag"] = inspections.results.apply(lambda x: 1 if x == "Fail" else 0)

### 2.2. Facility Risk Flags

In [ ]:
# Create risk flags
data["risk_1"] = inspections.results.apply(lambda x: 1 if x == "Risk 1 (High)" else 0)
data["risk_2"] = inspections.results.apply(lambda x: 1 if x == "Risk 2 (Medium)" else 0)
data["risk_3"] = inspections.results.apply(lambda x: 1 if x == "Risk 3 (Low)" else 0)

### 2.3. Violation Data

In [ ]:
# Load violation data
values = pd.read_csv(os.path.join(root_path, "DATA/violation_values.csv"))
counts = pd.read_csv(os.path.join(root_path, "DATA/violation_counts.csv"))

# Merge with violation data
data = pd.merge(data, values, on="inspection_id", how="left")
data = pd.merge(data, counts, on="inspection_id", how="left")

# Set default to 0
data.fillna(0, inplace=True)

### 2.4. Past Fails

In [ ]:
# Sort inspections by date
grouped = data.sort_values(by="inspection_date", inplace=True)

# Find previous inspections by shifting each license group
past_data = data.groupby("license").shift(1)

# Add past fails
data["past_fail"] = past_data.fail_flag

### 2.5. Past Violation Data

In [ ]:
# There's an issue with unfilled NAs

In [ ]:
# Select past violation values, remove past inspection id
past_values = past_data[values.columns].drop("inspection_id", axis=1).add_prefix("p")

# Set violation values to 0 for first inspections
past_values.fillna(0, inplace=True)

# Add past values to model data
data = data.join(past_values)

In [ ]:
# Add past violation counts
data["past_critical"] = past_data.critical_count
data["past_serious"] = past_data.serious_count
data["past_minor"] = past_data.minor_count

#### Time Since Last Inspection

In [ ]:
# Calculate time since previous inspection
deltas = pd.to_datetime(data.inspection_date) - pd.to_datetime(past_data.inspection_date)

# Add years since previous inspection
data["time_since_last"] = deltas.apply(lambda x: x.days / 365.25)

#### First Record Flag

In [ ]:
# Check if first record
data["first_record"] = data.time_since_last.map(lambda x: 1 if pd.isnull(x) else 0)

# Set time since last for first inspections to 2
data.time_since_last.fillna(2, inplace=True)

## 3. Business License Features

### 3.1. Matching Inspections with Licenses

In [ ]:
# Load business license data
licenses = pd.read_csv(os.path.join(root_path, "DATA/business_licenses.csv"))

In [ ]:
# Business licenses have numbers on end preventing simple match
# so using street number instead
def get_street_number(address):
    return address.split()[0]

licenses["street_number"] = licenses.address.apply(get_street_number)
inspections["street_number"] = inspections.address.apply(get_street_number)

In [ ]:
# Match based on DBA name and street number
venue_matches = pd.merge(inspections, licenses, left_on=["dba_name", "street_number"], right_on=["doing_business_as_name", "street_number"])

# Match based on license numbers
license_matches = pd.merge(inspections, licenses, left_on="license", right_on="license_number")

# Join matches, reset index, drop duplicates
matches = venue_matches.append(license_matches, sort=False)
matches.reset_index(drop=True, inplace=True)
matches.drop_duplicates(["inspection_id", "id"], inplace=True)

# Restrict to matches where inspection falls within license period
matches = matches.loc[matches.inspection_date.between(matches.license_start_date, matches.expiration_date)]

### 3.2. Filterering by Category

In [ ]:
# Select retail food establishment inspection IDs
retail = matches.loc[matches.license_description == "Retail Food Establishment", ["inspection_id"]]
retail.drop_duplicates(inplace=True)

# FILTER: ONLY CONSIDER INSPECTIONS MATCHED WITH RETAIL LICENSES
data = pd.merge(data, retail, on="inspection_id")

### 3.2. Calculating Age at Inspection

In [ ]:
# Convert dates to datetime format
matches.inspection_date = pd.to_datetime(matches.inspection_date)
matches.license_start_date = pd.to_datetime(matches.license_start_date)

def get_age_data(group):
    min_date = group.license_start_date.min()
    deltas = group.inspection_date - min_date
    group["age_at_inspection"] = deltas.apply(lambda x: x.days / 365.25)
    return group[["inspection_id", "age_at_inspection"]]

# Calculate (3 mins), drop duplicates
age_data = matches.groupby("license").apply(get_age_data).drop_duplicates()

In [ ]:
# Merge in age_at_inspection
data = pd.merge(data, age_data, on="inspection_id", how="left")

### 3.3. Calculating Category Data

In [ ]:
# Translate categories to snake-case titles
categories = {
    "Consumption on Premises - Incidental Activity": "consumption_on_premises_incidental_activity",
    "Tobacco": "tobacco",
    "Package Goods": "package_goods",
    "Limited Business License": "limited_business_license",
    "Outdoor Patio": "outdoor_patio",
    "Public Place of Amusement": "public_place_of_amusement",
    "Children's Services Facility License": "childrens_services_facility_license",
    "Tavern": "tavern",
    "Regulated Business License": "regulated_business_license",
    "Filling Station": "filling_station",
    "Caterer's Liquor License": "caterers_liquor_license",
    "Mobile Food License": "mobile_food_license"
}

# Create binary markers for license categories
def get_category_data(group):
    df = group[["inspection_id"]].iloc[[0]]
    for category in group.license_description:
        if category in categories:
            df[categories[category]] = 1
    return df
    
# group by inspection, get categories (2 mins)
category_data = matches.groupby("inspection_id").apply(get_category_data)

# Reset index, set absent categories to 0
category_data.reset_index(drop=True, inplace=True)
category_data.fillna(0, inplace=True)

In [ ]:
# Merge in category data
data = pd.merge(data, category_data, on="inspection_id", how="left")

## 4. Kernel Density Estimation

In [ ]:
# Load observation datasets
burglaries = pd.read_csv(os.path.join(root_path, "DATA/burglaries.csv"))
carts = pd.read_csv(os.path.join(root_path, "DATA/garbage_carts.csv"))
complaints = pd.read_csv(os.path.join(root_path, "DATA/sanitation_complaints.csv"))

In [ ]:
# Create datetime columns
inspections["datetime"] = pd.to_datetime(inspections.inspection_date)
burglaries["datetime"] = pd.to_datetime(burglaries.date)
carts["datetime"] = pd.to_datetime(carts.creation_date)
complaints["datetime"] = pd.to_datetime(complaints.creation_date)

In [ ]:
# FILTER: consider only inspections since 2012
# Otherwise early inspections have few/no observations within window
inspections = inspections.loc[inspections.inspection_date >= "2012"]

In [ ]:
from datetime import datetime, timedelta
from scipy import stats

def get_kde(observations, column_name, window, bandwidth):

    # Sort chronologically and index by datetime
    observations.sort_values("datetime", inplace=True)
    observations.index = observations.datetime.values
    
    # Generate kernel from 90 days of observations
    def get_kde_given_date(group):
        stop = group.datetime.iloc[0]
        start = stop - timedelta(days=window)
        recent = observations.loc[start:stop]
        
        x1 = recent.longitude
        y1 = recent.latitude
        values = np.vstack([x1, y1])
        kernel = stats.gaussian_kde(values)

        x2 = group.longitude
        y2 = group.latitude
        samples = np.vstack([x2, y2])
        group[column_name] = kernel(samples)
        return group[["inspection_id", column_name]]

    # Group inspections by date, generate kernels, sample
    return inspections.groupby("inspection_date").apply(get_kde_given_date)

In [ ]:
# Calculate burglary density estimates
burglary_kde = get_kde(burglaries, "burglary_kde", 90, 1)

# Calculate garbage cart density estimates
cart_kde = get_kde(carts, "cart_kde", 90, 1)

# Calculate sanitation complaint density estimates
complaint_kde = get_kde(complaints, "complaint_kde", 90, 1)

In [ ]:
# FILTER: only consider data since 2012 (with good kde data)
data = pd.merge(data, burglary_kde, on="inspection_id")
data = pd.merge(data, cart_kde, on="inspection_id")
data = pd.merge(data, complaint_kde, on="inspection_id")

## Weather

In [ ]:
# Load weather data
weather = pd.read_csv(os.path.join(root_path, "DATA/weather.csv"))

# Merge weather data with model data
data = pd.merge(data, weather, on="inspection_id")

## 3. Calculation of Model Data
We then set out derive all features originally considered by the Chicago team plus several novel features, translating from R to Python in the process. While relative weights in forecasting individual violations rather than number of critical violations may be different, we expect the significant databases and [NEEDS REPHRASING & STUFF]

[MAYBE HAVE BULLET POINTS FOR DATABASES, FEATURES]

Each row of the final training dataset represents an inspection, its outcomes, and a number of other features. [SOMETHING ABOUT TRAINING METHODOLOGY, MAYBE ABOUT FIGURING OUT WHICH FEATURES MATTER FOR WHICH]

We used the inspections database as the basis for our training data.

List of features is as follows:

### 3.1. features from food inspection data
[NEED TO WRITE THIS UP]
* used subset of inspections as basis
* merged with violation values & counts on inspection id
* created pass / fail flags
* sorted by date, shifted to get previous info
  * past fail
  * past critical, serious, minor
  * inspections with no past default to 0
* calculate time since last
  * binary first_record
  * time since last defaults to 2
  
### 3.2. features from business license data
[YET TO BE CALCULATED]
  
### 3.3. from heat maps
[YET TO BE CALCULATED]
  
### 3.4. from weather
[YET TO BE CALCULATED

## 4. Generating Model